In [1065]:
import pandas as pd
from transliterate import translit
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import re
import warnings
warnings.filterwarnings('ignore')

In [1066]:
# задать количество столбцов и строк отображения датафрейма
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [1067]:
# тема для графиков по умалчанию - plotly_white рекомендуется для презентаций
pio.templates.default = "plotly_white"

### Подготовить чистый датасет без пропусков по московским квартирам.
### Итог: 
 - файл data.csv, отвечающий критериям:
            названия колонок на английском языке в одно/несколько слов с нижним подчёркиванием
            в каждой колонке должны отсутствовать пропущенные значения (NaN, None и т.д.)
 - ноутбук preprocessing.ipynb, где показан процесс обработки данных


In [1068]:
# загружаем данные. 2 варианта с google диска или из локальной папки data и выодим на экран
# file_id = "130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7"
# path_data = f"https://drive.google.com/uc?export=download&id={file_id}"
path_data = r'../../data/_data.csv' # если не работает скачивание с google диска
data_df = pd.read_csv(path_data, encoding='utf-8', index_col=0)
data_df.head(5)

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...","+79660342340, +79099269384",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",+79852432860,Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


In [1069]:
# размер датафрейма
data_df.shape

(23368, 24)

In [1070]:
# оставляем только Москву
data_df = data_df[data_df['Адрес'].str.upper().str.contains('МОСКВА', na=False)].copy()
data_df.shape

(19737, 24)

Удаляем неинфомативные колонки, а также столбцы с большим количество пропусков.

К столбцам из предыдущего релиза(Серия дома, Название ЖК) добавим столбцы Телефоны, Ссылка на объявление, Описание, Тип

In [1071]:
# удаляем следующие колонки которые не несут ценности, или имеют количество пропусков (>70%)
col_drop = ['Серия дома', 'Название ЖК', 'Телефоны', 'Ссылка на объявление', 'Описание', 'Тип']
data_df = data_df.drop(col_drop, axis=1)
data_df.shape

(19737, 18)

In [1072]:
# статистика
data_df.describe(include='all')

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод
count,1.973700e+04,19202,19391,19737,19737,19737,8563,19737,17274,12509,13107,14587,17696,14822,19465,10535.000000,15545,11730
unique,NaN,24,4871,10452,9585,2415,5,2219,4,3462,18,3,20,3,488,NaN,34,2
top,NaN,1,м. Водный стадион (5 мин пешком),"Москва, Чапаевский переулок, 3",40.0/20.0/10.0,"3/5, Кирпичный",наземная,"40000.0 руб./ За месяц, Залог - 40000 руб., Ко...",Косметический,20,Балкон (1),Во двор,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,Пасс (1),Да
freq,NaN,6646,40,63,214,265,5283,1112,7361,1500,6336,9708,8500,8540,2553,NaN,4865,10262
mean,2.671149e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.992925,NaN,NaN
std,1.980106e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.852740,NaN,NaN
min,1.072986e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000,NaN,NaN
25%,2.712212e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.640000,NaN,NaN
50%,2.739284e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.640000,NaN,NaN
75%,2.746973e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.800000,NaN,NaN


In [1073]:
# выбираем пропущенные значения из датафрейма
pd.DataFrame((data_df.isna().mean() * 100) \
            .sort_values(ascending=False) \
            .round(2) \
            .reset_index()) \
            .rename(columns={
                    'index': 'Название столбца',
                    0: 'Процент пропусков, %'
})


,Название столбца,"Процент пропусков, %"
0,Парковка,56.61
1,"Высота потолков, м",46.62
2,Мусоропровод,40.57
3,"Площадь комнат, м2",36.62
4,Балкон,33.59
5,Окна,26.09
6,Можно с детьми/животными,24.90
7,Лифт,21.24
8,Ремонт,12.48
9,Санузел,10.34


13 признаков имеют пропуски, из которых 3 имеют количество пропусков более 40% значений.

Признак цена, в котором находится целевая переменная с купе с другой инфрмацией пропусков не имеет.

Следовательно можно предположить, что выделить целевую переменную в релизе 3 возможно в 100%.

Сложности по заполнению будут с полями, в которых отмечается значительное количество пропусков.

Не все можно заполнить стандартными средствами (mean, median, mode, const). Нужно поизучать.

`ID объявления` признак маркер, который не несет информации, но важен для определения цены другми подразделениями.

In [1074]:
data_df.head()

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет
2,271173086,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),NaN
3,272197456,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),NaN
4,273614615,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.9,"Пасс (1), Груз (1)",Да


In [1075]:
# выявим выбросы в высоте потолков, посмотрим количество
data_df[(data_df['Высота потолков, м'] < 2.4) | (data_df['Высота потолков, м'] > 8)]

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод
9,272900409,4,м. Арбатская (10 мин пешком),"Москва, улица Арбат, 30/3С1",90.0/66.0/10.0,"2/7, Сталинский",открытая,"210000.0 руб./ За месяц, Залог - 210000 руб., ...",Евроремонт,16 14 16 20,Балкон (1),На улицу и двор,"Совмещенный (1), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",310.0,Пасс (1),Да
247,271203810,1,м. Селигерская (None мин пешком),"Москва, Долгопрудная аллея, 14к1, ш. Дмитровск...",38.0/18.0/10.0,"3/14, Блочный",NaN,"32000.0 руб./ За месяц, Залог - 32000 руб., Ко...",Косметический,18,Балкон (1),На улицу и двор,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",260.0,"Пасс (1), Груз (1)",NaN
2969,274867097,3,м. Каширская (31 мин пешком),"Москва, Каширский проезд, 25к1",82.4,"12/22, Монолитно-кирпичный",наземная,"65000.0 руб./ За месяц, Залог - 75000 руб., Ко...",Евроремонт,NaN,Лоджия (1),На улицу,Совмещенный (2),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",28.0,"Пасс (2), Груз (1)",Нет
3759,274982211,"3, Изолированная",м. Домодедовская (20 мин пешком),"Москва, Борисовский проезд, 8К2",58.0/42.0/10.0,"6/9, Панельный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ср...",Косметический,18-12-12,Лоджия (1),На улицу и двор,Раздельный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Стиральная...",264.0,Пасс (1),NaN
4072,252733003,"2, Изолированная",м. Бабушкинская (6 мин на машине),"Москва, Полярная улица, 27к4",44.0/30.0/10.0,"7/25, Монолитно-кирпичный",наземная,"44000.0 руб./ За месяц, Залог - 44000 руб., Ко...",Евроремонт,16+14,NaN,Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",27.0,"Пасс (1), Груз (2)",NaN
4126,240892098,NaN,м. Медведково (25 мин пешком),"Москва, Полярная улица, 27к2",18.9/8.2/4.7,"7/15, Монолитный",наземная,"40000.0 руб./ За месяц, Залог - 40000 руб., Ко...",Косметический,NaN,NaN,На улицу,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Душевая ка...",2.3,"Пасс (1), Груз (1)",Нет
6616,274903162,NaN,м. Авиамоторная (9 мин пешком),"Москва, Красноказарменная улица, 14Ак1",21.0/14.0/2.0,"18/23, Монолитный",подземная,"48000.0 руб./ За месяц, Залог - 48000 руб., Ко...",Косметический,14,NaN,Во двор,Совмещенный (1),NaN,"Мебель в комнатах, Мебель на кухне, Душевая ка...",12.8,Пасс (1),NaN
7423,274946733,1,м. Нагатинская (8 мин пешком),"Москва, Варшавское шоссе, 49к1",37.0/15.0/8.0,"5/8, Блочный",наземная,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",Евроремонт,NaN,Балкон (1),На улицу,Совмещенный (1),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",26.0,Пасс (1),Да
7522,268545042,"6, Оба варианта",NaN,"Москва, Каширское шоссе, 16",132.0,"1/8, Кирпичный",наземная,"149999.0 руб./ За месяц, Залог - 149999 руб., ...",Дизайнерский,NaN,"Балкон (1), Лоджия (1)",На улицу,Раздельный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Душевая ка...",2.3,NaN,Нет
8343,274453264,2,м. Нагорная (12 мин пешком),"Москва, Криворожская улица, 1",41.0/26.0/6.0,"3/5, Кирпичный",NaN,"45000.0 руб./ За месяц, Залог - 45000 руб., Ко...",NaN,NaN,Балкон (1),На улицу,Раздельный (1),"Можно с детьми, Можно с животными","Ванна, Стиральная машина, Кондиционер, Интернет",2.3,NaN,NaN


Обычно высота квартир в пределах от 2.4 м (возможно до 2.1 с учетом перепланировки) и до 7.5 метров. Берем интервал высоты потолка от 2.0 по 8.0. Все остальное неверно и подлежит замене. 
Проверка высоты потолков в сети интернет показала, что значения:
- больше 200, это высота в сантиметрах(нужно разделить на 100)
- значения больше 20, но меньше 100 это высота деленая на 10 (вероятно забыли поставить точку)
- значения менее 2х невозможны или это исключения, которые единичны и не представляют ценности.

In [1076]:
data_df.head(6)

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет
2,271173086,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),NaN
3,272197456,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),NaN
4,273614615,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.9,"Пасс (1), Груз (1)",Да
5,274837728,3,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,NaN


In [1077]:
# удаляем выбросы
def height_room(x):
    if pd.isna(x):
        return x
    if x > 200:
        x /= 100
    if x > 22:
        x /= 10
    # Можно добавить проверку разумных границ
    if x < 2.0 or x > 8:  # например, всё вне [2.0, 6.0] — ошибка
        return np.nan  # или медиана, или None для последующей замены
    return x

data_df['Высота потолков, м'] = data_df['Высота потолков, м'].apply(height_room)
data_df[(data_df['Высота потолков, м'] < 2) | (data_df['Высота потолков, м'] > 8)]

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод


In [1078]:
# заполняем nan
def get_address(addr):
    if pd.isna(addr):
        return None
    addr_split = addr.split(',', maxsplit=3)
    if len(addr_split) >= 3:
        street = addr_split[1].strip()
        house = addr_split[2].strip()
        return f"{street}, {house}"
    else:
        return None

print(f"Доля пропусков  Высота потолков, м - {data_df['Высота потолков, м'].isna().mean() * 100}%")
# Применяем
data_df['address_tmp'] = data_df['Адрес'].apply(get_address)

data_df['Высота потолков, м'] = data_df.groupby('address_tmp')['Высота потолков, м'].transform(
    lambda x: x.fillna(x.median())
)
# выбираем пропущенные значения из датафрейма
print(f"Доля пропусков  Высота потолков, м - {data_df['Высота потолков, м'].isna().mean() * 100}%")

Доля пропусков  Высота потолков, м - 46.63322693418452%
Доля пропусков  Высота потолков, м - 23.711810305517556%


In [1079]:
data_df.head(2)

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,"Высота потолков, м",Лифт,Мусоропровод,address_tmp
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да,"улица Новый Арбат, 27"
1,271634126,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет,"улица Новый Арбат, 27"


Удалось корректно заполнить 23% пропусков по высоте
Остальное заполним медианой. Для реального проекта можно было бы еще выделить улицу, тип дома, серию дома.

In [1080]:
# заполняем оставшиеся nan высоты медианой
print(f"Доля пропусков  Высота потолков, м - {data_df['Высота потолков, м'].isna().mean() * 100}%")
data_df['Высота потолков, м'].fillna(data_df['Высота потолков, м'].median(), inplace=True)
print(f"Доля пропусков  Высота потолков, м - {data_df['Высота потолков, м'].isna().mean() * 100}%")

Доля пропусков  Высота потолков, м - 23.711810305517556%
Доля пропусков  Высота потолков, м - 0.0%


In [1081]:
print(f"Доля пропусков  Парковка - {data_df['Парковка'].isna().mean() * 100}%")
data_df['Парковка'] = data_df.groupby('address_tmp')['Парковка'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'не указано')
)
print(f"Доля пропусков  Парковка - {data_df['Парковка'].isna().mean() * 100}%")

Доля пропусков  Парковка - 56.61448041749%
Доля пропусков  Парковка - 0.0%


Признак Мусоропровод заполняем  по адресу модой, если такого адреса нет, то ставим не указано. Можно использовать моду, но количество пропусков значительно, а у Мусоропровода есть как плюсы так и минусы.

In [1082]:
data_df['Мусоропровод'].value_counts(normalize=True, dropna=False)

Мусоропровод
Да     0.519937
NaN    0.405685
Нет    0.074378
Name: proportion, dtype: float64

40% пропусков это много, чтобы заполнить модой

In [1083]:
print(f"Доля пропусков  Мусоропровод - {data_df['Мусоропровод'].isna().mean() * 100}%")
data_df['Мусоропровод'] = data_df.groupby('address_tmp')['Мусоропровод'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'не указано')
)
print(f"Доля пропусков  Мусоропровод - {data_df['Мусоропровод'].isna().mean() * 100}%")

Доля пропусков  Мусоропровод - 40.56847545219638%
Доля пропусков  Мусоропровод - 0.0%


Отработка балконов по аналогии с предыдущими, если есть информация по адресу, то ставим моду по ней, если нет, то общую моду

In [1084]:
def get_balkon(bal_tmp):
    mode_bal = bal_tmp.mode()
    if not mode_bal.empty:
        return bal_tmp.fillna(mode_bal.iloc[0])
    else:
        return bal_tmp.fillna('нет информации')

print(f"Доля пропусков  Балкон - {data_df['Балкон'].isna().mean() * 100}%")
data_df['Балкон'] = data_df.groupby('address_tmp')['Балкон'].transform(get_balkon)
print(f"Доля пропусков  Балкон - {data_df['Балкон'].isna().mean() * 100}%")

Доля пропусков  Балкон - 33.59173126614987%
Доля пропусков  Балкон - 0.0%


для корректного заполнения пропусков в количестве комнат и площади необходимо извлечь взаимно числовые значения для этих признаков.
и после по количеству комнат можно медианой заполнить площадь, а по площади  определить количество комнат
для заполнения площади:
- использовать группировку адрес + площадь квартир и взять медианное значение количества комнат округлив до целого, по остальным сгруппировать по площади и взять медианное значение

In [1085]:
#  выделение площади
def get_area_rent(text):
    if pd.isna(text) or str(text).strip() == '':
        return np.nan
    area_get = str(text).strip().split('/')[0]
    if area_get:
        try:
            return float(area_get)
        except:
            return np.nan
    return np.nan

data_df['area_tmp'] = data_df['Площадь, м2'].apply(get_area_rent)

In [1086]:
#  выделение количества комнат
def get_cnt_room_rent(text):
    if pd.isna(text) or str(text).strip() == '':
        return np.nan
    cnt_room_get = str(text).strip().split(',')[0]
    if cnt_room_get:
        try:
            return float(cnt_room_get)
        except:
            return np.nan
    return np.nan

data_df['count_room'] = data_df['Количество комнат'].apply(get_cnt_room_rent)

In [1087]:
print(f"Доля пропусков  Количество комнат - {data_df['Количество комнат'].isna().mean() * 100}%")
# Заполняем count_room по address + area_tmp
data_df['Количество комнат'] = data_df.groupby(['address_tmp', 'area_tmp'])['count_room'].transform(
    lambda x: x.fillna(x.median())
)

# Дополняем по area_tmp
data_df['Количество комнат'] = data_df.groupby('area_tmp')['count_room'].transform(
    lambda x: x.fillna(x.median())
)
print(f"Доля пропусков  Количество комнат - {data_df['Количество комнат'].isna().mean() * 100}%")


Доля пропусков  Количество комнат - 2.7106449815068148%
Доля пропусков  Количество комнат - 0.17733191467801593%


In [1088]:
# Остатки — глобальной медианой
print(f"Доля пропусков  Количество комнат - {data_df['Количество комнат'].isna().mean() * 100}%")
global_median_room = data_df['count_room'].median()
data_df['Количество комнат'] = data_df['Количество комнат'].fillna(global_median_room)
print(f"Доля пропусков  Количество комнат - {data_df['Количество комнат'].isna().mean() * 100}%")

Доля пропусков  Количество комнат - 0.17733191467801593%
Доля пропусков  Количество комнат - 0.0%


In [1089]:
# заполним столбец полными значениями для использования далее
data_df['count_room'] = data_df['Количество комнат'].apply(get_cnt_room_rent)

Признак ремонт заполним значением не указано. Невозможно предположить какой ремонт в наличии.

In [1090]:
print(f"Доля пропусков  Ремонт - {data_df['Ремонт'].isna().mean() * 100}%")
data_df['Ремонт'] = data_df['Ремонт'].fillna('не указано')
print(f"Доля пропусков  Ремонт - {data_df['Ремонт'].isna().mean() * 100}%")

Доля пропусков  Ремонт - 12.479100167198661%
Доля пропусков  Ремонт - 0.0%


Признак лифт группировкой по адресу + заполнить модой, остальное заполнить просто Пасс (1), лифт есть практически во всех домах в Москве, кроме 5этажек старых.

In [1091]:
def get_lift_by_address(lift_val):
    mode_val = lift_val.mode()
    if not mode_val.empty:
        return lift_val.fillna(mode_val.iloc[0])
    else:
        return lift_val

print(f"Доля пропусков  Лифт - {data_df['Лифт'].isna().mean() * 100}%")
# Заполняем по адресу
data_df['Лифт'] = data_df.groupby('address_tmp')['Лифт'].transform(get_lift_by_address)
print(f"Доля пропусков  Лифт - {data_df['Лифт'].isna().mean() * 100}%")

Доля пропусков  Лифт - 21.23929675229265%
Доля пропусков  Лифт - 13.193494452044382%


In [1092]:
# Оставшиеся NaN заполняем значением не указано и в Релизе 3 можем с ними поработать
print(f"Доля пропусков  Лифт - {data_df['Лифт'].isna().mean() * 100}%")
data_df['Лифт'] = data_df['Лифт'].fillna('не указано')   
print(f"Доля пропусков  Лифт - {data_df['Лифт'].isna().mean() * 100}%")

Доля пропусков  Лифт - 13.193494452044382%
Доля пропусков  Лифт - 0.0%


по лифтам есть работа для Релиза 3, для этажей <= 5 установить что лифта нет, для > 5 что есть

для корректного заполнения пропусков в признаке санузел.
- использовать группировку адрес + количество комнат и взять моду, по остальным сгруппировать по количеству комнат и взять моду

In [1093]:
def get_san(san_val):
    mode_val = san_val.mode()
    if not mode_val.empty:
        return san_val.fillna(mode_val.iloc[0])
    return san_val

print(f"Доля пропусков  Санузел - {data_df['Санузел'].isna().mean() * 100}%")
# Заполняем по адресу + количеству комнат
data_df['Санузел'] = data_df.groupby(['address_tmp', 'count_room'])['Санузел'].transform(get_san)

# Заполняем по количеству комнат
data_df['Санузел'] = data_df.groupby('count_room')['Санузел'].transform(get_san)
print(f"Доля пропусков  Санузел - {data_df['Санузел'].isna().mean() * 100}%")

Доля пропусков  Санузел - 10.340983938795157%
Доля пропусков  Санузел - 0.0%


остальное заполняем значением есть. В Москве предполагается наличие санузла в квартирах.

In [1094]:
data_df['Санузел'] = data_df['Санузел'].fillna('есть')
print(f"Доля пропусков  Санузел - {data_df['Санузел'].isna().mean() * 100}%")

Доля пропусков  Санузел - 0.0%


Признак Можно с детьми/животными заполняем не указано. Так как это индивидуальное пожелание арендодателя.

In [1095]:
print(f"Доля пропусков  Можно с детьми/животными - {data_df['Можно с детьми/животными'].isna().mean() * 100}%")
data_df['Можно с детьми/животными'] = data_df['Можно с детьми/животными'].fillna('не указано')
print(f"Доля пропусков  Можно с детьми/животными - {data_df['Можно с детьми/животными'].isna().mean() * 100}%")

Доля пропусков  Можно с детьми/животными - 24.90246744692709%
Доля пропусков  Можно с детьми/животными - 0.0%


Для запонения признака Окна логично использовать группировку адрес + количество комнат и заполнить модой, остальное сгруппировать по количеству комнат и заполнить модой

In [1096]:
def get_wind(wind_val):
    mode_val = wind_val.mode()
    if not mode_val.empty:
        return wind_val.fillna(mode_val.iloc[0])
    return wind_val

print(f"Доля пропусков  Окна - {data_df['Окна'].isna().mean() * 100}%")
# Заполняем по адресу + количеству комнат
data_df['Окна'] = data_df.groupby(['address_tmp', 'count_room'])['Окна'].transform(get_wind)

# Заполняем по количеству комнат
data_df['Окна'] = data_df.groupby('count_room')['Окна'].transform(get_wind)
print(f"Доля пропусков  Окна - {data_df['Окна'].isna().mean() * 100}%")

Доля пропусков  Окна - 26.093124588336625%
Доля пропусков  Окна - 0.005066626133657597%


In [1097]:
def get_wind(wind_val):
    mode_val = wind_val.mode()
    if not mode_val.empty:
        return wind_val.fillna(mode_val.iloc[0])
    return wind_val

print(f"Доля пропусков  Окна - {data_df['Окна'].isna().mean() * 100}%")
# Заполняем по адресу + площадь
data_df['Окна'] = data_df.groupby(['address_tmp', 'area_tmp'])['Окна'].transform(get_wind)

# Заполняем по площади
data_df['Окна'] = data_df.groupby('area_tmp')['Окна'].transform(get_wind)
print(f"Доля пропусков  Окна - {data_df['Окна'].isna().mean() * 100}%")

Доля пропусков  Окна - 0.005066626133657597%
Доля пропусков  Окна - 0.0%


In [1098]:
# Оставшиеся NaN заполняем значением не указано
print(f"Доля пропусков  Окна - {data_df['Окна'].isna().mean() * 100}%")
data_df['Окна'] = data_df['Окна'].fillna('не указано')   
print(f"Доля пропусков  Окна - {data_df['Окна'].isna().mean() * 100}%")

Доля пропусков  Окна - 0.0%
Доля пропусков  Окна - 0.0%


признак метро заполнить группировкой по адресу и взять моду. по остальным проставить не указано и поработать в релизе 3

In [1099]:
def get_metro(metro_val):
    mode_val = metro_val.mode()
    if not mode_val.empty:
        return metro_val.fillna(mode_val.iloc[0])
    return metro_val

print(f"Доля пропусков  Метро - {data_df['Метро'].isna().mean() * 100}%")
# Заполняем по количеству комнат
data_df['Метро'] = data_df.groupby('address_tmp')['Метро'].transform(get_metro)
print(f"Доля пропусков  Метро - {data_df['Метро'].isna().mean() * 100}%")

Доля пропусков  Метро - 1.7530526422455288%
Доля пропусков  Метро - 1.008258600597862%


In [1100]:
# Оставшиеся NaN заполняем значением не указано и проработаем в Релизе 3
print(f"Доля пропусков  Метро - {data_df['Метро'].isna().mean() * 100}%")
data_df['Метро'] = data_df['Метро'].fillna('не указано')   
print(f"Доля пропусков  Метро - {data_df['Метро'].isna().mean() * 100}%")

Доля пропусков  Метро - 1.008258600597862%
Доля пропусков  Метро - 0.0%


Для признака Дополнительно использовать группировку ремонт + количество комнат, втак как для квартир дизайнерского ремонта будет схожее наполнение, как и для евро ремонта, если остались пропуски только ремонт

In [1101]:
def get_dop(val):
    mode_val = val.mode()
    if not mode_val.empty:
        return val.fillna(mode_val.iloc[0])
    return val

print(f"Доля пропусков  Дополнительно - {data_df['Дополнительно'].isna().mean() * 100}%")
# По ремонту + количеству комнат
data_df['Дополнительно'] = data_df.groupby(['Ремонт', 'count_room'])['Дополнительно'].transform(get_dop)

# По типу ремонта
data_df['Дополнительно'] = data_df.groupby('Ремонт')['Дополнительно'].transform(get_dop)

print(f"Доля пропусков  Дополнительно - {data_df['Дополнительно'].isna().mean() * 100}%")

Доля пропусков  Дополнительно - 1.3781223083548666%
Доля пропусков  Дополнительно - 0.0%


In [1102]:
print(f"Доля пропусков  Дополнительно - {data_df['Дополнительно'].isna().mean() * 100}%")
# Остатки — 'не указано'
data_df['Дополнительно'] = data_df['Дополнительно'].fillna('не указано')
print(f"Доля пропусков  Дополнительно - {data_df['Дополнительно'].isna().mean() * 100}%")

Доля пропусков  Дополнительно - 0.0%
Доля пропусков  Дополнительно - 0.0%


Признак Площадь комнат, м2 несет информацию, подобную признаку Площадь, м2, при это имеет 36% пропусков

In [1103]:
def get_area_room(val):
    mode_val = val.mode()
    if not mode_val.empty:
        return val.fillna(mode_val.iloc[0])
    return val

print(f"Доля пропусков  Площадь комнат, м2 - {data_df['Площадь комнат, м2'].isna().mean() * 100}%")
# Заполняем по адресу + количеству комнат
data_df['Площадь комнат, м2'] = data_df.groupby(['address_tmp', 'count_room'])['Площадь комнат, м2'].transform(get_area_room)

# Заполняем по количеству комнат
data_df['Площадь комнат, м2'] = data_df.groupby('count_room')['Площадь комнат, м2'].transform(get_area_room)
print(f"Доля пропусков  Площадь комнат, м2 - {data_df['Площадь комнат, м2'].isna().mean() * 100}%")

Доля пропусков  Площадь комнат, м2 - 36.621573694077114%
Доля пропусков  Площадь комнат, м2 - 0.005066626133657597%


In [1104]:
def get_area_room2(wind_val):
    mode_val = wind_val.mode()
    if not mode_val.empty:
        return wind_val.fillna(mode_val.iloc[0])
    return wind_val

print(f"Доля пропусков  Площадь комнат, м2 - {data_df['Площадь комнат, м2'].isna().mean() * 100}%")
# Заполняем по адресу + количеству комнат
data_df['Площадь комнат, м2'] = data_df.groupby(['address_tmp', 'area_tmp'])['Площадь комнат, м2'].transform(get_area_room2)

# Заполняем по количеству комнат
data_df['Площадь комнат, м2'] = data_df.groupby('area_tmp')['Площадь комнат, м2'].transform(get_area_room2)
print(f"Доля пропусков  Площадь комнат, м2 - {data_df['Площадь комнат, м2'].isna().mean() * 100}%")

Доля пропусков  Площадь комнат, м2 - 0.005066626133657597%
Доля пропусков  Площадь комнат, м2 - 0.0%


In [1105]:
data_df.columns

Index(['ID  объявления', 'Количество комнат', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'Парковка', 'Цена', 'Ремонт', 'Площадь комнат, м2', 'Балкон',
       'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно',
       'Высота потолков, м', 'Лифт', 'Мусоропровод', 'address_tmp', 'area_tmp',
       'count_room'],
      dtype='object')

удаляем временные столбцы data_df = data_df.drop(['house_address', 'area', 'rooms_count'], axis = 1)

In [1106]:
data_df = data_df.drop(['address_tmp', 'area_tmp', 'count_room'], axis = 1)

In [1107]:
data_df.columns

Index(['ID  объявления', 'Количество комнат', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'Парковка', 'Цена', 'Ремонт', 'Площадь комнат, м2', 'Балкон',
       'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно',
       'Высота потолков, м', 'Лифт', 'Мусоропровод'],
      dtype='object')

In [1108]:
def make_rus_lat(text):
    text = translit(text, 'ru', reversed=True)
    text = re.sub(r'[^\w\s]', '', text)
    text = '_'.join(text.split())
    return text

data_df.columns = [make_rus_lat(col).lower() for col in data_df.columns]

In [1109]:
data_df.columns

Index(['id_objavlenija', 'kolichestvo_komnat', 'metro', 'adres', 'ploschad_m2',
       'dom', 'parkovka', 'tsena', 'remont', 'ploschad_komnat_m2', 'balkon',
       'okna', 'sanuzel', 'mozhno_s_detmizhivotnymi', 'dopolnitelno',
       'vysota_potolkov_m', 'lift', 'musoroprovod'],
      dtype='object')

In [1110]:
new_name_col = [
    'ad_id',
    'rooms',
    'metro',
    'address',
    'total_area',
    'building_type',
    'parking',
    'price',
    'renovation_type',
    'room_areas',
    'balcony',
    'windows_view',
    'bathroom_type',
    'pets_allowed',
    'amenities',
    'ceiling_height',
    'elevator',
    'garbage_chute'
]

In [1111]:
data_df.columns = new_name_col

In [1112]:
data_df.columns

Index(['ad_id', 'rooms', 'metro', 'address', 'total_area', 'building_type',
       'parking', 'price', 'renovation_type', 'room_areas', 'balcony',
       'windows_view', 'bathroom_type', 'pets_allowed', 'amenities',
       'ceiling_height', 'elevator', 'garbage_chute'],
      dtype='object')

In [1113]:
# выбираем пропущенные значения из датафрейма
pd.DataFrame((data_df.isna().mean() * 100) \
            .sort_values(ascending=False) \
            .round(2) \
            .reset_index()) \
            .rename(columns={
                    'index': 'Название столбца',
                    0: 'Процент пропусков, %'
})


,Название столбца,"Процент пропусков, %"
0,ad_id,0.0
1,rooms,0.0
2,metro,0.0
3,address,0.0
4,total_area,0.0
5,building_type,0.0
6,parking,0.0
7,price,0.0
8,renovation_type,0.0
9,room_areas,0.0


In [1114]:
data_df.head()

,ad_id,rooms,metro,address,total_area,building_type,parking,price,renovation_type,room_areas,balcony,windows_view,bathroom_type,pets_allowed,amenities,ceiling_height,elevator,garbage_chute
0,271271157,4.0,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,Лоджия (1),На улицу и двор,Раздельный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.0,"Пасс (4), Груз (1)",Да
1,271634126,4.0,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,Лоджия (1),На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.5,"Пасс (1), Груз (1)",Нет
2,271173086,4.0,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,25 25 20 25,Лоджия (1),На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),Да
3,272197456,4.0,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,14-42-20-19,Балкон (1),На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2,Пасс (1),Да
4,273614615,2.0,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",наземная,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,20 18,"Балкон (1), Лоджия (1)",На улицу и двор,Совмещенный (2),не указано,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.9,"Пасс (1), Груз (1)",Да


Названия столбцов переименованы на английский в стиле snake_case.

В каждой колонке должны отсутствовать пропущенные значения (NaN, None и т.д.)


In [1115]:
data_df.describe()

,ad_id,rooms,ceiling_height
count,1.973700e+04,19737.000000,19737.000000
mean,2.671149e+08,2.004155,2.779694
std,1.980106e+07,1.003646,0.258546
min,1.072986e+08,1.000000,2.000000
25%,2.712212e+08,1.000000,2.640000
50%,2.739284e+08,2.000000,2.700000
75%,2.746973e+08,3.000000,2.800000
max,2.750064e+08,6.000000,6.000000


Сохраняем файл в католог релиза 2

In [ ]:
path = r'../../realises/release_2_0/'
data_df.to_csv(path+'data.csv', index=False)